In [2]:
import os
from dotenv import load_dotenv

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [13]:
artist = pd.read_csv("./processed/selina/artists.csv").drop(["genres"], axis=1)["artist_uri"].to_frame()
print(len(artist), "artists")

18439 artists


In [14]:
tracks = pd.read_csv("./processed/selina/tracks_features.csv")["spotify_track_uri"].to_frame()
print(len(tracks), "tracks")

32644 tracks


In [15]:
artist_track = pd.read_csv("./processed/selina/artist_track.csv")
print(len(artist_track), "artist_track entries")

53086 artist_track entries


In [85]:
missing_tracks = pd.DataFrame(artist_track[~artist_track["track_uri"].isin(all_uris)]["track_uri"].unique(), columns=["track_uri"])
len(missing_tracks)

4406

Looks like 4406 track_uris, that are present in the artist_track dataframe, do not exist in the track dataframe.
This is most likely caused by assigning a random uri from all uris, that point to the same track.

In [74]:
tracks_all_uris = pd.read_csv("./processed/selina/tracks_all_uris.csv")
tracks_all_uris.head(3)

,master_metadata_track_name,master_metadata_album_album_name,spotify_track_uri,master_metadata_album_artist_name
0,Womanizer,Circus (Deluxe Version),['spotify:track:4fixebDZAVToLbUCuEloa2'],Britney Spears
1,Boom Boom Pow,THE E.N.D. (THE ENERGY NEVER DIES),"['spotify:track:3oDFtOhcN08qeDPAK6MEQG', 'spot...",Black Eyed Peas
2,Shut Up And Drive,Good Girl Gone Bad: Reloaded,['spotify:track:4Tn2llBm1g0UlWctmgPL8Z'],Rihanna


In [75]:
tracks_all_uris["spotify_track_uri"] = tracks_all_uris["spotify_track_uri"].replace("\[|\]|\'", "", regex=True).str.split(",")

In [81]:
all_uris = sum(tracks_all_uris["spotify_track_uri"].tolist(), [])

In [ ]:
all_uris